# 第4章 朴素贝叶斯

前两章，我们要求分类器做出艰难决策，但是分类器又实惠产生错误结果。这时我们可以要求分类器给出一个最优的类别猜测结果，同时给出这个猜测的概率估计值。

在本章从最简单的**概率分类器**开始，然后给出一些假设来学习**朴素贝叶斯分类器**。这里的朴素是指：整个形式化过程只做最原始、最简单的假设。

## 4.1 基于贝叶斯决策的分类方法

> 朴素贝叶斯
>
> - 优点：在数据较少的情况下仍然有效，可以处理多类别问题。
> - 缺点：对于输入数据的准备方式较为敏感。
> - 适用类型：标称型数据(一般在有限的数据中取，而且只存在'是'和'否'两种不同的结果)。

朴素贝叶斯，是贝叶斯决策理论的一部分，那么什么是贝叶斯决策理论呢？

对于新数据，代入根据已知数据计算出来的概率函数，取各概率函数结果最大的为新数据的分类。

## 4.2 条件概率

对于一个事件$A$，它在统计学上有以下与概率相关的事件：

- 随机事件$A$发生的**先验概率**$P(A)$：

  $P(A)=\frac{\#(A)}{\#}$

- 随机事件$A$和随机事件$B$同时发生的**联合概率**：

  $P(A,B)=P(A|B)·P(B)=P(B|A)·P(A)$

  当$A B$相互独立，即$P(A|B)=P(A)$时，$P(A,B)=P(A)·P(B)$

- 给定事件$B$已发生的条件下，$A$发生的**条件概率**：

  $P(A|B)=\frac{P(A,B)}{P(B)}$

> 以3B1B视频中的提到的为例：
>
> 已知有农民200人，图书馆管理员10人。其中图书馆管理员中文质彬彬的人群有4人，农民中有20人。那么请问已知证据（Evidence）：一个人是文质彬彬的性格，那么假设（Hypothesis）“这个人是图书馆管理员的概率”$P(h|E)$是多少？
>
>  我们可以直接得出的结论有：
>
> 一个人是图书管理员的先验概率(prior)：$P(H)=\frac{10}{10+200}=\frac{1}{21}$
>
> 一个人文质彬彬的先验概率：$P(E)=\frac{4+20}{10+200}$
>
> 文质彬彬时，是管理员的条件概率：$P(H|E)=\frac{4}{4+20}$
>
> 文质彬彬时，是农民的条件概率：$P(\neg H|E)=\frac{20}{4+20}$
>
> ---
>
> 基于上面的信息，继续做的推论有：
>
> 既是管理员，又文质彬彬的联合概率：$P(H,E)=\frac{4}{210}$
>
> 既文质彬彬又是管理员的联合概率：$P(E,H)=$

[扩展阅读](https://0x100.club/math/bayes.html)

总结：

联合概率，在样本空间不变的情况下，使发生的条件更加苛刻了（如果联合的事件完全独立，那么就变成了不可能事件；）缩小了分子的取值，所以概率会变低；

条件概率，限制了事件发生的样本空间，所以概率既可能变大也可能变小。

 由联合概率公式，我们可以推出贝叶斯公式：
$$
P(A|B)=\frac{P(B|A)·P(A)}{P(B)}
$$

## 4.3 使用条件概率分类

根据贝叶斯公式，我们对于输入的信息$X$，判断其对应的分类，可得：
$$
P(Y|X)=\frac{P(X|Y)·P(Y)}{P(X)}=\frac{P(X|Y)}{P(X)}·P(Y)
$$

## 4.4 朴素贝叶斯进行分类

朴素贝叶斯中的朴素（native）一词，是指：样本中的特征都是相互独立的，不考虑特征之间的相互影响。且每个特征同等重要。

## 4.5 算法实现
### 4.5.1 OneHot编码处理

In [5]:
from numpy import *

# 返回数据集
def loadDataSet():
    postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    #1 代表脏话, 0 not
    classVec = [0,1,0,1,0,1]    
    return postingList,classVec
        
# 对数据集去重，并返回新的数据集
def createVocabList(dataSet):
    vocabSet = set([])  #create empty set
    for document in dataSet:
        vocabSet = vocabSet | set(document) #union of the two sets
    return list(vocabSet)

# 接收词汇表 和要编码的数据，返回编码后的结果；是词袋模型，统计了每个词出现的次数
def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
        else: 
            print(f"the {word}: %s is not in my Vocabulary!")
    return returnVec

In [6]:
# 加载数据集和标签
list_posts,list_classes = loadDataSet()
# 返回不重复的词列表
list_vocab = createVocabList(list_posts)
print(list_vocab)

['take', 'dalmation', 'food', 'him', 'how', 'problems', 'dog', 'flea', 'to', 'park', 'cute', 'worthless', 'buying', 'has', 'stupid', 'my', 'stop', 'ate', 'maybe', 'is', 'mr', 'please', 'steak', 'help', 'not', 'posting', 'licks', 'I', 'quit', 'garbage', 'love', 'so']


In [7]:
list_mat = []
for line in list_posts:
    # 将数据集的每一行按照list_vocab，转化为向量
    list_mat.append(setOfWords2Vec(list_vocab,line))
np_mat = array(list_mat)
print(f"""此时的数据集格式已经变成:
{np_mat}""")

此时的数据集格式已经变成:
[[0 0 0 0 0 1 1 1 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0]
 [1 0 0 1 0 0 1 0 1 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0]
 [0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 1 1 1 0 0 1 0 1 0 0 0 1 0 0 0 0 0]
 [0 0 1 0 0 0 1 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]]


至此，我们的准备工作已经完成。根据贝叶斯公式，我们需要计算几个概率，代码如下:

todo：待补充要计算的概率是什么，以及推导过程

In [21]:

def trainNB0(trainMatrix,trainCategory):
    """
    输入文档矩阵trainMatrix 和对应的标签向量trainCategory；
    返回onehot编码后的各个位置上的，给定是1或者0的各个单词的条件概率的对数
    """
    # 样本数量
    numTrainDocs = len(trainMatrix)
    # 每个样本的长度
    numWords = len(trainMatrix[0])
    
    # 求出了负面评价的先验概率
    pAbusive = sum(trainCategory)/float(numTrainDocs)
    
    # 声明与样本长度一致的全1 array，用来累加某个词出现的频率
    # 全1是避免：某个词出现次数为0，求条件概率时为0，导致联合概率也为0
    # 这也称为拉普拉斯平滑
    p0Num = ones(numWords); p1Num = ones(numWords)
    # 同样也是为了避免概率为0
    p0Denom = 2.0; p1Denom = 2.0 
    
    for i in range(numTrainDocs):
        # 条件概率，限制分母的
        if trainCategory[i] == 1:
            # 会在全为1的OneHot向量上，做向量加法，即各自激活的位置上+1
            p1Num += trainMatrix[i]
            # 累加当前OneHot上词的数量，即样本空间的总数。
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    # 将条件概率乘法，通过对数函数转换为先ln再加法，避免因为数据小，精度丢失。
    # 是向量类型，贝叶斯公式右边的P(B|A)
    p1Vect = log(p1Num/p1Denom)          
    p0Vect = log(p0Num/p0Denom)          
    return p0Vect,p1Vect,pAbusive

In [22]:
# 返回OneHot向量上，给定条件是积极的或者消极的，下不同词出现的频率
vec_p0,vec_p1,pro_abusive = trainNB0(list_mat, list_classes)
print(vec_p0)
print(vec_p1)
print(pro_abusive)

[-3.25809654 -2.56494936 -2.56494936 -3.25809654 -3.25809654 -2.56494936
 -2.56494936 -2.56494936 -2.56494936 -3.25809654 -2.56494936 -2.56494936
 -3.25809654 -2.56494936 -3.25809654 -3.25809654 -2.56494936 -1.87180218
 -2.56494936 -2.56494936 -2.56494936 -2.56494936 -3.25809654 -2.56494936
 -2.56494936 -3.25809654 -2.56494936 -2.15948425 -3.25809654 -2.56494936
 -3.25809654 -2.56494936]
[-1.65822808 -3.04452244 -3.04452244 -2.35137526 -2.35137526 -3.04452244
 -2.35137526 -3.04452244 -2.35137526 -2.35137526 -3.04452244 -3.04452244
 -2.35137526 -3.04452244 -2.35137526 -2.35137526 -3.04452244 -3.04452244
 -3.04452244 -3.04452244 -3.04452244 -3.04452244 -2.35137526 -3.04452244
 -3.04452244 -2.35137526 -3.04452244 -2.35137526 -2.35137526 -1.94591015
 -1.94591015 -3.04452244]
0.5


In [25]:
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    # vec2Classify是OneHot后的待预测数据，做乘法，可以理解为将OneHot上为1的位置，激活对应的条件概率
    # 做加法是因为 条件概率 是取Ln的，所以其实就是假定各自独立的联合概率分布
    p1 = sum(vec2Classify * p1Vec) + log(pClass1)    #element-wise mult
    p0 = sum(vec2Classify * p0Vec) + log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else: 
        return 0

In [28]:
def testingNB():
    listOPosts,listClasses = loadDataSet()
    myVocabList = createVocabList(listOPosts)
    trainMat=[]
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    p0V,p1V,pAb = trainNB0(array(trainMat),array(listClasses))
    
    testEntry = ['love', 'my', 'dalmation']
    thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry,'classified as: ',classifyNB(thisDoc,p0V,p1V,pAb))
    
    testEntry = ['stupid', 'garbage']
    thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry,'classified as: ',classifyNB(thisDoc,p0V,p1V,pAb))

    testingNB()

['love', 'my', 'dalmation'] classified as:  0
['stupid', 'garbage'] classified as:  1


## 4.6 本章小结

本章学习了朴素贝叶斯的分类器，首先是要假定样本的特征都是相互独立的，这也是“朴素”一词的意思。

以垃圾邮件分类为例：
1. 根据已有样本，首先对文本One-Hot编码
2. 然后可以计算已知是垃圾邮件或者非垃圾邮件的条件下，每个单词出现的频率向量$v$（为了避免出现0，要假定各个单词最少出现一次，避免精度丢失，所以取log）
3. 将未知数据One-Hot编码后，乘以$v$，对结果求和，并加上$log(P(A))$（因为取了对数，这两个加法其实都是乘法）
4. 在代码中，其实还少了要除以$P(B)$的步骤，因为不论是求哪种情况，都需要除以$P(B)$，所以代码中省略了，对结果没有影响。即我不需要计算未知样本的概率。